## Building out a model

In [321]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier



In [322]:
model_features = pd.read_csv("C:/Users/HP/OneDrive/Documents/DANNY DATA/Customer Support Data/model_data.csv")

In [323]:
columns_to_drop = ["category", "sub-category", "issue_reported_at", "issue_responded", "response_time_(minutes)", 
                   "response_time_(seconds)", "time_of_issue", "time_of_response", "agent_name", "supervisor"]

model_features.drop(columns=columns_to_drop, axis=1, inplace=True)
model_features.head()

,channel_name,response_time_(hours),report_day_name,report_month_day,response_month_day,response_day_name,manager,tenure_bucket,agent_shift,csat_score
0,Outcall,0.566667,Tuesday,1,1,Tuesday,Jennifer Nguyen,On Job Training,Morning,5
1,Outcall,0.033333,Tuesday,1,1,Tuesday,Michael Lee,>90,Morning,5
2,Inbound,0.366667,Tuesday,1,1,Tuesday,William Kim,On Job Training,Evening,5
3,Inbound,0.333333,Tuesday,1,1,Tuesday,John Smith,>90,Evening,5
4,Inbound,0.033333,Tuesday,1,1,Tuesday,Michael Lee,0-30,Morning,5


In [324]:
model_features.columns

Index(['channel_name', 'response_time_(hours)', 'report_day_name',
       'report_month_day', 'response_month_day', 'response_day_name',
       'manager', 'tenure_bucket', 'agent_shift', 'csat_score'],
      dtype='object')

In [325]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()

In [326]:
scale_features = ["response_time_(hours)", "report_month_day", "response_month_day"]

# one-hot encoding, no inhherent order
model_features = pd.get_dummies(model_features, columns=["channel_name", "manager", "agent_shift", "report_day_name", "response_day_name"])

# Inherent  order is present
model_features["tenure_bucket"] = label_encoder.fit_transform(model_features["tenure_bucket"])

In [327]:
model_features.columns

Index(['response_time_(hours)', 'report_month_day', 'response_month_day',
       'tenure_bucket', 'csat_score', 'channel_name_Email',
       'channel_name_Inbound', 'channel_name_Outcall', 'manager_Emily Chen',
       'manager_Jennifer Nguyen', 'manager_John Smith', 'manager_Michael Lee',
       'manager_Olivia Tan', 'manager_William Kim', 'agent_shift_Afternoon',
       'agent_shift_Evening', 'agent_shift_Morning', 'agent_shift_Night',
       'agent_shift_Split', 'report_day_name_Friday', 'report_day_name_Monday',
       'report_day_name_Saturday', 'report_day_name_Sunday',
       'report_day_name_Thursday', 'report_day_name_Tuesday',
       'report_day_name_Wednesday', 'response_day_name_Friday',
       'response_day_name_Monday', 'response_day_name_Saturday',
       'response_day_name_Sunday', 'response_day_name_Thursday',
       'response_day_name_Tuesday', 'response_day_name_Wednesday'],
      dtype='object')

In [328]:
model_features["csat_binary"] = model_features["csat_score"].apply(lambda x: 1 if x > 3.7 else 0)
model_features.drop(columns=["csat_score"], axis=1, inplace=True)

In [329]:
X = model_features.drop(columns=["csat_binary"], axis=1)
y = model_features["csat_binary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

